In [1]:
from llama3 import Llama
import torch
from pathlib import Path
import json
from llama3.model import ModelArgs, Transformer
from llama3.tokenizer import ChatFormat, Dialog, Message, Tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, PeftModel, PeftConfig
from trl import SFTTrainer
from datasets import load_dataset, Dataset, Features, ClassLabel, Value

In [2]:
ckpt_dir = "F:\AugustRoboticsLLM\Meta-Llama-3-8B-Instruct"
tokenizer_path = "F:\AugustRoboticsLLM\Meta-Llama-3-8B-Instruct"
output_dir = "F:\AugustRoboticsLLM\LoRAAdapter"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
peft_config = LoraConfig(
    r=32,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj"],
)

In [3]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, trust_remote_code=True, add_bos_token=False)
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token

In [9]:
class_names = ["Symptom", "Solution"]
features = Features({name: Value('string') for name in class_names})
QandA_dataset = load_dataset("csv", data_dir="F:\AugustRoboticsLLM\dataset", sep=',', quoting=1, quotechar=r'"', doublequote=True, features=features)

Generating train split: 0 examples [00:00, ? examples/s]

C:\Users\myper\.conda\envs\ml\lib\site-packages\datasets\download\streaming_download_manager.py:784: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


In [11]:
def prepare_dataset(dataset, tokenizer=tokenizer):
    # prompting eng: https://github.com/meta-llama/llama-recipes/blob/main/recipes/quickstart/Prompt_Engineering_with_Llama_3.ipynb
    dialogs = [
        [
            {"role": "system", "content": "Your role is an on-site robotics operation engineer who gives technical and practical advice to client who use Lionel robot to draw marks on the exhibition.\n"
                                       "You belong to August Robotics Ltd. Please do not answer any question not relate to our business, you can simply refuse it by saying no.\n"
                                       "Let's think through this carefully, step by step.\n"},
            {"role": "user", "content": f"{dataset['Symptom']}"},
            {"role": "assistant", "content": f"{dataset['Solution']}"}
        ],
    ]
    # print(type(tokenizer.apply_chat_template(dialogs, tokenize=False)[0]))
    return {"formattedchat": tokenizer.apply_chat_template(dialogs, tokenize=False)[0]}

QandA_dataset = QandA_dataset.map(prepare_dataset, remove_columns=('Symptom', 'Solution'))
# QandA_dataset = QandA_dataset.train_test_split(test_size=0.1)
QandA_dataset

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['formattedchat'],
        num_rows: 32
    })
})

In [12]:
print(QandA_dataset["train"]["formattedchat"][0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Your role is an on-site robotics operation engineer who gives technical and practical advice to client who use Lionel robot to draw lines on the ground.
You belong to August Robotics Ltd. Please do not answer any question not relate to our business, you can simply refuse it by saying no.
Let's think through this carefully, step by step.<|eot_id|><|start_header_id|>user<|end_header_id|>

Nothing is displayed when a map is viewed on the tablet<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Start by refreshing the webpage on your tablet and navigate back to the map. Confirm that the map has not been archived. If the problem persists, access the client portal at https://portal.augustrobotics.com and verify that the map is still active.<|eot_id|>


In [4]:
text_dataset = load_dataset("text", data_dir="F:\AugustRoboticsLLM\/text_dataset")
print(text_dataset)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 515
    })
})


In [5]:
model = AutoModelForCausalLM.from_pretrained(ckpt_dir, quantization_config=bnb_config, device_map="auto")
model.resize_token_embeddings(len(tokenizer))
model = prepare_model_for_kbit_training(model)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
del model
model = AutoModelForCausalLM.from_pretrained(ckpt_dir, 
    return_dict=True,
    torch_dtype=torch.float16, device_map="auto")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the disk and cpu.


In [13]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    num_train_epochs=1,
    optim="paged_adamw_32bit",
    save_steps=200,
    logging_steps=5,
    learning_rate=0.0001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=False,
    lr_scheduler_type="cosine",
    dataloader_num_workers=0
)

trainer = SFTTrainer(
    model=model,
    train_dataset=QandA_dataset["train"],
    peft_config=peft_config,
    dataset_text_field="formattedchat",
    # dataset_text_field="text",
    dataset_batch_size=1,
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
    
)

trainer.train()
trainer.model.save_pretrained(output_dir)

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

C:\Users\myper\.conda\envs\ml\lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
C:\Users\myper\.conda\envs\ml\lib\site-packages\torch\utils\checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
5,3.866200
10,2.348400
15,1.234800
20,1.303400
25,1.028900
30,1.259500


In [4]:
base_model = AutoModelForCausalLM.from_pretrained(
    ckpt_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    device_map="cpu",
    offload_buffers=False,
)
model = PeftModel.from_pretrained(base_model, "F:\AugustRoboticsLLM\LoRAAdapter", device_map="cpu")
model = model.merge_and_unload()
model.save_pretrained("F:\AugustRoboticsLLM\ARLLM")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [17]:
generation_config = GenerationConfig.from_pretrained(ckpt_dir)
generation_config.max_new_tokens=150
generation_config.repetition_penalty = 1.1
prompts = [
        # For these prompts, the expected answer is the natural continuation of the prompt
        "The Guiding Station is designed for",
    ]
model_inputs = tokenizer(prompts, padding=True, truncation=True, return_tensors="pt", max_length=512)
output = model.generate(**model_inputs, generation_config=generation_config)
print(tokenizer.decode(output[0], skip_special_tokens=False))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>The Guiding Station is designed for use with our Guiding Station Base Plate (sold separately) and allows you to attach your own custom guiding system, such as a string or a laser line, to help guide the router during operation. The station includes two adjustable arms that can be positioned to accommodate various router sizes and styles.
Please note that this item requires the purchase of the Guiding Station Base Plate (Part #1013-01). The base plate must be purchased and installed before using the Guiding Station. Contact customer service if you have any questions about compatibility or installation. This product is not intended for use on routers that do not have a flat bottom or sides. Please ensure your router has a suitable surface area for attachment before purchasing. Consult your router's manual for further information


In [ ]:
generation_config = GenerationConfig.from_pretrained(ckpt_dir)
generation_config.max_new_tokens=150
generation_config.repetition_penalty = 1.1
model_inputs = tokenizer(tokenizer.apply_chat_template([
    {
    "role": "system", "content": "Your role is an on-site robotics operation engineer who gives technical and practical advice to client who use Lionel robot to draw lines on the ground.\n"
    "You belong to August Robotics Ltd. Please do not answer any question not relate to our business, you can simply refuse it by saying no.\n"
    "Let's think through this carefully, step by step.\n"
    """As a member of the support team, it is our responsibility to ensure our Lease Clients have a great experience every time they use their Lionel system. It is not uncommon for them to run into issues when operating the robots - some of these issues may be caused by human error, or system error. Being on the support team means you are able to identify the issue and provide advice to solve the issue. Being on support doesn't mean you have to stare at live client deployments all day, but it does mean you need to be attentive to support requests/emails/messages that come through - and reply to these requests as quickly as possible.
One of the most difficult things to do while on support is to identify the root cause of a problem. Often, we are helping clients very far away, without being able to see the equipment. In many situations, remote access is our best tool. By using remote access, we can effectively “see” what the client sees - at least on their tablet. We can also perform a deeper analysis on certain issues by viewing the logs. However, remote access, and checking logs is not always the solution, and often we need to consider various factors to solve a problem.
1.Identify the problem symptoms: The first step of solving a problem is to identify exactly what the problem is. Often, we will need to ask the client a few questions to get a better understanding, and narrow down the problem area. Some of these questions could include: Is this the first time this issue has occurred? How often does this issue occur? Did the user do anything specific to get the problem to occur? Has the user restarted any equipment?
2.Attempt to match the issue to our list of common issues: Once we have asked a few questions, and narrowed down the issue, we should check our table of common troubleshooting steps. Many client issues of the past have been tracked, evaluated and stored in this table. There is no need to try to re-invent a solution if we already have them. 
3.If the cause of issue is still unknown: If the issue is still not solved, we need to attempt to identify the cause. Sometimes the cause can be separated into hardware or software. But the problems are not always clear. Depending on the problem, we need to determine a possible approach for the solution. Some possible approaches we have are: Asking the user to reboot problematic devices. Using remote access to view the MSP GUI. Using remote access to view the logs of the Map Server/Robot. If for example, their robot's nozzle is blocked, and they aren't sure what to do - it would be silly to use remote access to assist them. In a situation like this, we need to provide steps that deal with the problem at the source, such as checking the bottle connection, checking that the pump is working, replacing the nozzle etc.
4.Knowing when to escalate a problem: As a member of support - you are not alone. Ideally, each member of support should be capable of solving 95% of issues clients face. There will always be undiscovered issues or new scenarios. It's important to know when to escalate a problem to the broader support team. Escalate a problem when: No solution has been found within ~15 minutes. None of the steps in the troubleshooting table have worked. You have tried alternative solutions, that have not worked.
Things to consider when being on support: Language / tone of our replies: As a company, we have an extremely international team. We speak many different languages - and so do our clients. Most of our clients will be able to communicate in English. Perfect English is not expected from you! For many of us, English is our second or third language. It might be important to make the client aware of this - a quick introductory message will go a long way. For example: “Hi Mr/Ms ****, Thank you for your message, my name is Leo, I am part of the August Robotics team in Shenzhen. I am here to assist you with your issue. Please note - English is not my first language, but I will try my best to communicate clearly.”
Additionally, we need to ensure we are as respectful as possible to the client. Many of our clients are not technically skilled, so something that may seem simple to us as engineers might be very difficult to understand for them. We need to exercise patience when trying to get information about a problem from the clients.
Quality of our Replies: When we reply to clients about issues they are having, we need to think about the quality of reply we are giving. Short answers may not be a good way of communicating. We should try to provide as much detail as possible - it's sometimes a good idea to pretend the client has never seen the system before. \n"""
    
    },
    {
        "role": "user", "content": "what is the guiding station?"
    }], tokenize=False), return_tensors="pt").to("cuda")
print(model_inputs)
output = model.generate(**model_inputs, generation_config=generation_config)
print(tokenizer.decode(output[0], skip_special_tokens=False))

In [4]:
model = AutoModelForCausalLM.from_pretrained("F:\AugustRoboticsLLM\ARLLM", quantization_config=bnb_config, device_map="auto")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
import gc
del model
gc.collect()

3719

In [31]:
del model

In [14]:
gc.collect()

72

In [16]:
model.unload()

NameError: name 'model' is not defined